In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

In [148]:
import random
import sys
import time
import glob
import os
import time
import pandas as pd
import IPython
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import pandas as pd
import librosa
import librosa.display
import seaborn as sns
from sklearn.preprocessing import normalize
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from surfboard.feature_extraction import extract_features
from surfboard.sound import Waveform
from sklearn.utils import shuffle

In [160]:
import pandas as pd
reduce_data = pd.read_csv('/content/gdrive/MyDrive/csv-files/combined_csv.csv')
noisy_data= pd.read_csv('/content/gdrive/MyDrive/csv-files/combined_csv_noisy.csv')

In [161]:
reduce_data = reduce_data.iloc[:,13:26]
noisy_data = noisy_data.iloc[:,13:26]
preprocessed_batsounds = reduce_data.values.tolist()
preprocessed_noisesounds = noisy_data.values.tolist()

In [162]:


batsounds_normalized =normalize(np.array(preprocessed_batsounds),axis=1)
noisesounds_normalized = normalize(np.array(preprocessed_noisesounds),axis=1)
df_batsounds = pd.DataFrame(batsounds_normalized)
df_batsounds['target']=1
df_noisesounds = pd.DataFrame(noisesounds_normalized)
df_noisesounds['target'] =0
train_data = pd.concat([df_batsounds,df_noisesounds])
train_data = shuffle(train_data)
X = train_data.drop(['target'],axis=1).values
y = train_data['target'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
LEN_SOUND = len(preprocessed_batsounds[0])
NUM_CLASSES = 2 # Bat or no bat

model = Sequential()
model.add(Dense(128, activation='relu',input_shape=(LEN_SOUND,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(2))
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
model.summary()
t0=time.time()
history = model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=100,shuffle=True) 
print("training time:", round(time.time()-t0, 3))
t1=time.time()
y_pred = np.argmax(model.predict(X_test), axis=1)
print("predict time:", round(time.time()-t1, 3))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
# Plot accuracy per iteration
plt.plot(history.history['accuracy'], label='acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt

matrix = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=[1,0])
print('Confusion matrix : \n',matrix)

tp, fn, fp, tn = confusion_matrix(y_true=y_test, y_pred=y_pred,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix = classification_report(y_test,y_pred,labels=[1,0])
print('Classification report : \n',matrix)

In [ ]:

soundarray, sr = librosa.load("/content/PinkPanther60.wav")
maxseconds = int(len(soundarray)/sr)
count =1
for second in range(maxseconds-1):
  audiosample = np.array(soundarray[second*sr:(second+1)*sr])
  sound = Waveform(signal=audiosample,sample_rate=sr)
  feature_temp = extract_features([sound], ['mfcc', 'rms'], ['mean', 'std'])
  feature_temp =feature_temp.iloc[:,13:26]
  feature_temp = feature_temp.values.tolist()
  testdata = normalize(np.array(feature_temp),axis=1)
  prediction = model.predict(testdata)

  if np.argmax(prediction) ==1:
      print("Detected a bat at " + str(second) + " out of " + str(maxseconds) + " seconds")
      print(prediction)
      count=count+1
print("total detected bat seconds:\t"+str(count) + "\tout of\t"+str(maxseconds)+"\tseconds") 